In [14]:
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role

In [15]:
role = get_execution_role()

In [67]:
bucketName = 'signet-ring-cell-bryan'
folderName = 'challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/'

In [46]:
s3_client = boto3.client('s3')
bucket = s3_client.list_objects(Bucket=bucketName, Prefix=folderName)

In [47]:
# First five images (xml files) in our dataset
for img_num in range(1,11):
    if img_num%2==0:
        print(bucket['Contents'][img_num]['Key'])

challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-59515-2003-2010.xml
challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-34589-61706-2030-2044.xml
challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-36515-58465-2013-2071.xml
challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-37528-60747-2016-2000.xml
challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-38368-62991-2040-2016.xml


In [48]:
len(bucket['Contents'])

155

In [19]:
# import pickle

# s3client = boto3.client('s3')
# s3 = boto3.resource('s3')
# src_bucket = s3.Bucket(bucketName)
# data_key = 'challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-59515-2003-2010.xml'
# data_location = 's3://{}/{}'.format(bucketName, data_key)
# response = s3client.get_object(Bucket=bucketName, Key=folderName)
# files = src_bucket.objects.all()
# first_file = s3.Object(bucketName, data_key)

In [25]:
import s3fs
fs = s3fs.S3FileSystem()

# To List 5 files in your accessible bucket
fs.ls('signet-ring-cell-bryan/challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/')[:5]

# open it directly
with fs.open('s3://signet-ring-cell-bryan/challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-59515-2003-2010.xml') as f:
        data = f.read() 

In [26]:
from bs4 import BeautifulSoup 
import json

In [52]:
def xml_to_json(data, file_name):
    Bs_data = BeautifulSoup(data, "xml") 
    zz=Bs_data.find_all("size")
    #print(zz)
    yyyy=str(zz)
    size=yyyy.partition('\n')[2]
    #print(size)
    wid=size.split("</width>",1)[0] 
    wid=wid.split("<width>",1)[1]
    #print(wid)
    hei=size.split("</height>",1)[0] 
    hei=hei.split("<height>",1)[1]
    #print(hei)
    depth=size.split("</depth>",1)[0] 
    depth=depth.split("<depth>",1)[1]
    #print(depth)
    b_unique = Bs_data.find_all('bndbox') 
    data1={}
    data1['files']=file_name
    data1['image_size']=[]
    data1['image_size'].append({
        'width' : wid,
        'height' : hei,
        'depth' : depth
    })
    data1['annotations']=[]
    for i in range(len(b_unique)):
        z=b_unique[i]
        y=str(z)
        yy=y.partition('\n')[2]
        xmin=yy.split("</xmin>",1)[0]
        xmin=xmin.split("<xmin>",1)[1]
        ymin=yy.split("</ymin>",1)[0] 
        ymin=ymin.split("<ymin>",1)[1]
        xmax=yy.split("</xmax>",1)[0] 
        xmax=xmax.split("<xmax>",1)[1]
        ymax=yy.split("</ymax>",1)[0] 
        ymax=ymax.split("<ymax>",1)[1]
        left=int(xmin)
        top=int(ymax)
        height=str(int(ymax)-int(ymin))
        width=str(int(xmax)-int(xmin))
        data1['annotations'].append({
            'class_id': 0,
            'left': left,
            'top' : top,
            'width' : width,
            'height' :height
        })
    data1['categories']=[]
    data1['categories'].append({
        'class_id' : 0,
        'name' : "signet ring cell"
    })
    

#s3='signet-ring-cell-bryan/challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/'
#json.dump_s3(data1, "data.json") # saves json to s3://bucket/key
    with open('test.json', 'w') as outfile:
        print("writing file")
        json.dump(data1, outfile) # saves json to s3://bucket/key

In [53]:
xml_to_json(data, 's3://signet-ring-cell-bryan/challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-59515-2003-2010.xml')


writing file


In [51]:
import os

In [73]:
def json_to_s3(bucket, folderName, fileName):
    with open(fileName, 'rb') as f:
        s3_client.upload_fileobj(f, bucket, 'challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/test1.json')
    #except ClientError as e:
      #  print(e)

In [74]:
json_to_s3(bucketName, folderName, 'test.json')

In [75]:
# Create our bucket for training and validation data

prefix = 'src_detection_model/'

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'
train_annotation_channel = prefix + '/train_annotation'
validation_annotation_channel = prefix + '/validation_annotation'

In [76]:
sess = sagemaker.Session()
od_bucket = sess.default_bucket()